## Результаты проведенных экспериментов
Эксперимент проводился на двух моделях: функциональной и последовательной.
Обучение модели  при (transfer learning) без и частичной тонкой настройкой дали не хорошие результаты.
С увеличением размера image_shape улучились результаты прогноза.

Использованы transfer learning с fine-tuning, в последовательной модели включен Batch Normalization слой, при различных image_shape. Применили разные функции callback Keras.

Использованы библиотеки Albumentations для аугментации изображений и TTA (Test Time Augmentation).  

В основном обучение проводилось в 5 эпох, за иключением  одного случая для Xception (12 эпох).
Предобученная сеть Xception для Функциональной Модели без тонкой настройкой.Результаты:
* Точность модели составила около 39% при image_shape = (90,120);
* Точность модели составила около 50% при image_shape = (150,150);
* Точность модели составила около 60% при image_shape = (224,224):
* Точность модели составила около 70% при image_shape = (320,320);

Предобученная сеть Xception для Функциональной Модели с тонкой настройкой:
* Точность модели составила около 94,76% при image_shape = (224,224);
* Результат модели  составила 95,88%  при image_shape = (320,320) за 12 эпох.

Предобученная сеть EfficientNetB0 с тонкой настройкой точность  Функциональной Модели 25.59% при image_shape = (320,320)

Предобученная сеть EfficientNetB3 с тонкой настройкой для последовательной модели составила 11.94% при image_shape = (320,320). Большая амплитуда зиг-зага Accuracy для каждой эпохи.

Предобученная сеть EfficientNetB5 для Функциональной Модели при image_shape = (320,320) за 5 эпох:
* без тонкой настройкой точность составила 92.92%.
* с тонкой настройкой точность составила 94,89%. Почему-то Accuracy зиг-заг по эпохам, не понятно?!
    

#### Проведенные вычислительные эксперименты показали, что из всех моделей более перспективными оказались две базовые модели: EfficientNetB5 и Xception с тонкой настройкой.

Последовательная модель с тонкой настройкой за 5 эпох:
*  с базовой моделью EfficientNetB5 точность предсказания достигли  94.93%. Наблюдается не устойччивость точности предсказания.
*  с базовой моделью Xception точность предсказания достигли 93.09%.

Последовательная модель с тонкой настройкой за 9 эпох с базовой моделью Xception:
* с базовой моделью Xception при image_shape = (320,320): Accuracy: 96.69%.

Последовательная модель с тонкой настройкой за 9 эпох с базовой моделью EfficientNetB5:
* при image_shape = (320,320): Accuracy: 96.87%; 96.57%;
* при image_shape = (240,240): Accuracy:  95.84%.



#### Для окончательного расчета выбрана последовательная модель с базовой моделью EfficientNetB5 с тонкой настройкой.
Использована стратегия с последовательным увеличением image_shape и эпох:
* в последовательную модель добавлен слой Batch Normalization;
* размер картинки изменили: 
     image_shape = (320,320); 
     image_shape = (320,420); 
     image_shape = (512,512);
* настройка LR, optimizer, loss;
* использован функции callback:  *добавили lr_scheduler (экспоненциально уменьшать скорость через 2 эпохи);  *добавили раннюю остановку; *добавили функцию контрольной точки, чтобы сохранить лучшую модель;
* аугментация данных с помощью Albumentations (два варианта)
* добавлено TTA (Test Time Augmentation) для прогноза.

Обучение произведено в двух шагах следующим образом:
* обучаем  с image_shape = (320,420) и с аугментацией широким набором параметров; 
* выбераем обученные параметры лучшей модели итерации, т.е. best_model.hdf5 ;
* следующим шагом загружем в модель параметры лучшей модели предыдущего шага и обучаем  модели с увеличением до image_shape = (512,512) с включением  двух параметровой аугментацией изображений.

In [1]:
!nvidia-smi

In [2]:
# Обновление tensorflow
# !pip install tensorflow --upgrade
# Загрузка модели efficientnet
!pip install -q efficientnet
# Загружаем обвязку под keras для использования продвинутых библиотек аугментации, например, albuminations
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor update tensorflow

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os


import tensorflow as tf
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.regularizers import l2
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.efficientnet import EfficientNetB0, EfficientNetB5, EfficientNetB3
from tensorflow.keras.layers import *
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
                                   
# import efficientnet.tfkeras as efn

from sklearn.model_selection import train_test_split, StratifiedKFold

import PIL
from PIL import ImageOps, ImageFilter
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

In [5]:
print(os.listdir("../input/sf-dl-car-classification"))

In [6]:
!pip freeze > requirements.txt

# Основные настройки

In [7]:
# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.

EPOCHS               = 7 # эпох на обучение
BATCH_SIZE           = 8 # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-4
VAL_SPLIT            = 0.15 # сколько данных выделяем на тест = 15%

CLASS_NUM            = 10  # количество классов в нашей задаче
H_IMG_SIZE           = 420 # Горизонтальный размер рисунка
V_IMG_SIZE           = 320  # Вертикалбный размер рисунка
IMG_CHANNELS         = 3   # у RGB 3 канала
input_shape          = (V_IMG_SIZE, H_IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'
PATH = "../working/car/" # рабочая директория

In [8]:
# Устаналиваем конкретное значение random seed для воспроизводимости
os.makedirs(PATH,exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)  
PYTHONHASHSEED = 0

In [9]:
class_names = [
  'Приора', #0
  'Ford Focus', #1
  'Самара', #2
  'ВАЗ-2110', #3
  'Жигули', #4
  'Нива', #5
  'Калина', #6
  'ВАЗ-2109', #7
  'Volkswagen Passat', #8
  'ВАЗ-21099' #9
]

In [10]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/sf-dl-car-classification/"+data_zip,"r") as z:
        z.extractall(PATH)
        
print(os.listdir(PATH))

In [11]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")

## Конец подготовки данных

# EDA / Анализ данных

In [12]:
train_df.head()

In [13]:
train_df.info()

In [14]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [15]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12,8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3,3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index])+'  '+class_names[random_image_cat[index]])
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [16]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

from skimage import io 
def imshow(image_RGB): 
    io.imshow(image_RGB) 
    io.show()

### Для вывода кривых training_acc и vol_accuracy,  training_loss и vol_loss

In [17]:
# Для вывода кривых training_acc и vol_accuracy,  training_loss и vol_loss
def learning_graphic(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
 
    epochs = range(len(acc))
 
    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
 
    plt.figure()
 
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
    return plt
# learning_graphic(history).show()

## Аугментация данных с помощью Albumentations 

In [18]:
import albumentations as a
from ImageDataAugmentor.image_data_augmentor import *

In [19]:
augmentations = a.Compose([
# добавляем размытие по Гауссу и шум с вероятностью 50%
      a.Blur(p=0.05), 
      a.GaussNoise(p=0.05),
    #  Увеличим вероятности сдвига, масштабирования, поворота, т.к. по умолчанию их значения низкие. 
    a.ShiftScaleRotate(shift_limit=0.0625, 
                       scale_limit=0.01, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=20, 
                       p=.75), 

    a.RandomBrightness(limit=0.2, p=0.5),
    
# Добавим еще несколько аугментации с параметрами по умолчанию 
# модель автомобиля может отличаться в зависимости от зеркального отображения,
# мы можем включить  NO VERTICAL FLIP,  т.к. автомобили всегда находятся в горизонтальном положении
    
    a.HorizontalFlip(),
    a.HueSaturationValue(), # случайный оттенок и насыщенность
    a.RGBShift(),
    a.FancyPCA(alpha=0.1,  always_apply=False,  p=0.5),
    a.Resize(V_IMG_SIZE, H_IMG_SIZE),
    
#  добавить OneOfs с вероятностью по умолчанию 50% для яркостного контраста
    a.OneOf([
        a.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        a.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)],
        p=0.5)
 ])

### Генерация данных

In [20]:
# Подготовка для генерации данных:
datagen = ImageDataAugmentor(
                        rescale=1./255,
                        augment=augmentations, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT)

# Генерация данных:

train_datagen = datagen.flow_from_directory(
    PATH+'train/',      # директорий
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='training') # обучающие данные

test_datagen = datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # данные проверки

# Используем для тестовых данных  Test Time Augmentation (TTA)
test_gen = ImageDataAugmentor(rescale=1./255) # rescale=1./255 
test_sub_generator = test_gen.flow_from_dataframe(dataframe=sample_submission,
                                            directory=PATH+'test_upload/',
                                            x_col="Id",
                                            y_col=None,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(V_IMG_SIZE, H_IMG_SIZE),
                                            batch_size=BATCH_SIZE)

### Проверка генерированных данных train_generator, test_generator

In [21]:
x,y = train_datagen.next()
print('Пример картинок из train_generator')
plt.figure(figsize=(12,8))

for i in range(0,8):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    plt.title('Class: '+str(y[i]))
    plt.axis('off')
plt.show()

In [22]:
x,y = test_datagen.next()
print('Пример картинок из test_generator')
plt.figure(figsize=(12,8))

for i in range(0,6):
    image = x[i]
    plt.subplot(3,3, i+1)
    plt.imshow(image)
    plt.title('Class: '+str(y[i]))
    plt.axis('off')
plt.show()

# Построение модели

In [23]:
base_model = EfficientNetB5(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

In [24]:
# Set new head
model = Sequential()
model.add(base_model)

# Add pooling layer
model.add(GlobalAveragePooling2D())

# Add a fully-connected layer
model.add(Dense(256, 
                      activation='relu', 
                      bias_regularizer=l2(1e-4),
                      activity_regularizer=l2(1e-5)))

# Add batch normalization
model.add(BatchNormalization())
model.add(Dropout(0.25)) # and dropout
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

# And a final layer for 10 classes
model.add(Dense(CLASS_NUM, activation='softmax'))

# This is the model we will train
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), metrics=["accuracy"])

In [25]:
len(model.trainable_variables)

In [26]:
# Add функцию контрольной точки, чтобы сохранить лучшую модель
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1,
                             mode = 'max')

# Добавим lr_scheduler (экспоненциально уменьшать скорость через 2 эпохи)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2, # уменьшим lr в 5 раз
                              patience=2, # если нет улучшения через 2 эпохи - уменьшить lr

                              min_lr=0.0000001,
                              verbose=1,
                              mode='auto')

# Добавим раннюю остановку
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          patience = 3,
                          restore_best_weights = True)

callbacks_list = [checkpoint, earlystop, lr_scheduler]

## Step 1

In [27]:
history = model.fit(
        train_datagen,
        steps_per_epoch = len(train_datagen),
        validation_data = test_datagen, 
        validation_steps = len(test_datagen),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [28]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
# model.save('../working/model_step_1.hdf5')
model.load_weights('best_model.hdf5')

In [29]:
scores = model.evaluate_generator(test_datagen, steps=len(test_datagen), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [30]:
learning_graphic(history).show()

### Предсказание на тестовых данных: Step 1

In [31]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_datagen.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [32]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_step_1.csv', index=False)
print('Save submit')

# Использован Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [33]:
submission.head()

### Конец Step 1

## Step 2

In [34]:
EPOCHS               = 8   # эувеличиваем эпох на обучение
BATCH_SIZE           = 2    # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR                   = 1e-5
VAL_SPLIT            = 0.15 # 
H_IMG_SIZE           = 512  # Горизонтальный размер рисунка
V_IMG_SIZE           = 512  # Вертикалбный размер рисунка
IMG_CHANNELS         = 3
input_shape          = (V_IMG_SIZE, H_IMG_SIZE, IMG_CHANNELS)

### Генерация данных

In [35]:
# Двух параметровая аугментация изображений (правильно ли так назвать?)
augmentations = a.Compose([
     a.ShiftScaleRotate(shift_limit=0.0625, 
                       scale_limit=0.01, 
                       interpolation=1, 
                       border_mode=4, 
                       rotate_limit=20, 
                       p=.75),
    a.HorizontalFlip(p=0.5)
])

In [36]:
 # Подготовка для генерации данных:
datagen = ImageDataAugmentor(                       
                        # rescale=1./255,
                        augment=augmentations, 
                        seed=RANDOM_SEED,
                        validation_split=VAL_SPLIT)

# Генерация данных:

train_datagen = datagen.flow_from_directory(
    PATH+'train/',      # директорий
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, 
    subset='training') # обучающие данные

test_datagen = datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation') # данные проверки


# Используем для тестовых данных  Test Time Augmentation (TTA) (datagen(augmentations))
test_sub_generator = datagen.flow_from_dataframe(dataframe=sample_submission,
                                            directory=PATH+'test_upload/',
                                            x_col="Id",
                                            y_col=None,
                                            shuffle=False,
                                            class_mode=None,
                                            target_size=(V_IMG_SIZE, H_IMG_SIZE),
                                            batch_size=BATCH_SIZE)

In [37]:
# Set new head
model = Sequential()
model.add(base_model)

# Add pooling layer
model.add(GlobalAveragePooling2D())

# Add a fully-connected layer
model.add(Dense(256, 
                      activation='relu', 
                      bias_regularizer=l2(1e-4),
                      activity_regularizer=l2(1e-5)))

# Add batch normalization
model.add(BatchNormalization())
model.add(Dropout(0.25)) # and dropout
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

# And a final layer for 10 classes
model.add(Dense(CLASS_NUM, activation='softmax'))

# This is the model we will train
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=LR), 
              metrics=["accuracy"])

In [38]:
model.load_weights('best_model.hdf5')

In [39]:
# Add функцию контрольной точки, чтобы сохранить лучшую модель
checkpoint = ModelCheckpoint('best_model.hdf5', 
                             monitor = ['val_accuracy'], 
                             verbose = 1,
                             mode = 'max')

# Добавим lr_scheduler (экспоненциально уменьшать скорость через 3 эпохи)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2, # уменьшим lr в 5 раз
                              patience=2, # если нет улучшения через 2 эпохи - уменьшить lr

                              min_lr=0.0000001,
                              verbose=1,
                              mode='auto')

# Добавим раннюю остановку
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          patience = 3,
                          restore_best_weights = True)

callbacks_list = [checkpoint, earlystop, lr_scheduler]

In [40]:
history = model.fit(
        train_datagen,
        steps_per_epoch = len(train_datagen),
        validation_data = test_datagen, 
        validation_steps = len(test_datagen),
        epochs = EPOCHS,
        callbacks = callbacks_list
)

In [41]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last_Effi.hdf5')
model.load_weights('best_model.hdf5')

In [42]:
scores = model.evaluate_generator(test_datagen, steps=len(test_datagen), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [43]:
learning_graphic(history).show()

### Конец Step 2 

# Предсказание на тестовых данных:  Step 2

In [44]:
test_sub_generator.samples

In [45]:
test_sub_generator.reset()
predictions = model.predict_generator(test_sub_generator, steps=len(test_sub_generator), verbose=1) 
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_datagen.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]

In [46]:
filenames_with_dir=test_sub_generator.filenames
submission = pd.DataFrame({'Id':filenames_with_dir, 'Category':predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/','')
submission.to_csv('submission_Effi.csv', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [47]:
submission.head()

Clean PATH
import shutil
shutil.rmtree(PATH)

from keras.models import load_model
model = load_model('model.h5')

predictions = 0.6*model_1.predict(sub_generator) \
    + 0.25*model_2.predict(sub_generator) \
    + 0.15*model_3.predict(sub_generator)
predictions = predictions.argmax(axis=1)
predictions

## Интересно, к какому классу модель отнесет вот эти автомобили?

Советы:
* Примените transfer learning с fine-tuning
* Настройте LR, optimizer, loss
* Подберите другие переменные (размер картинки, батч и т.д.)
* Попробуйте и другие архитектуры сетей (а не только Xception) или их ансамбли. Примеры SOTA на ImageNet
* Добавьте Batch Normalization и поэкспериментируйте с архитектурой “головы”
* Примените другие функции callback Keras https://keras.io/callbacks/
* Добавьте TTA (Test Time Augmentation)
* Дополнительно*: Используйте разные техники управления Learning Rate (https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng) http://teleported.in/posts/cyclic-learning-rate/ (eng))
* Дополнительно*: Добавьте более продвинутые библиотеки аугментации изображений (например, Albumentations )

### Удачи в соревновании!